In [ ]:
# Know Your Fan - Sistema de Perfil para os Fãs
# =============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import json
import os
import base64
from io import BytesIO
from IPython.display import HTML, display, Image as IPImage
import ipywidgets as widgets
from ipywidgets import FileUpload, Button, VBox, HBox, Output, Text, Label
import cv2
import pytesseract
from PIL import Image
import io
import re

class SimpleNERModel:
    def __call__(self, text):
        entities = []
        words = text.split()
        
        for i, word in enumerate(words):
            if re.match(r'\d{3}\.\d{3}\.\d{3}-\d{2}', word) or re.match(r'\d{11}', word):
                entities.append({'word': word, 'entity': 'B-CPF'})
            
            elif word[0].isupper() and i > 0 and not words[i-1].endswith('.'):
                if i > 0 and words[i-1][0].isupper():
                    entities.append({'word': word, 'entity': 'I-PER'})
                else:
                    entities.append({'word': word, 'entity': 'B-PER'})
            
            elif any(addr in word.lower() for addr in ['rua', 'avenida', 'av.', 'alameda', 'praça']):
                entities.append({'word': word, 'entity': 'B-LOC'})
                for j in range(1, 4):
                    if i+j < len(words):
                        entities.append({'word': words[i+j], 'entity': 'I-LOC'})
        
        return entities

class SimpleFaceModel:
    def __call__(self, image):
        return [{'box': [10, 10, 100, 100], 'confidence': 0.95}]

class FanProfile:
    def __init__(self):
        self.profile_data = {
            'personal_info': {},
            'esports_interests': {},
            'events_attended': [],
            'purchases': [],
            'social_media': {},
            'esports_profiles': {},
            'documents': {}
        }
    
    def collect_personal_info(self, name, address, cpf, birth_date, email, phone):
        self.profile_data['personal_info']['name'] = name
        self.profile_data['personal_info']['address'] = address
        self.profile_data['personal_info']['cpf'] = cpf
        self.profile_data['personal_info']['birth_date'] = birth_date
        self.profile_data['personal_info']['email'] = email
        self.profile_data['personal_info']['phone'] = phone
        return "Informações pessoais coletadas com sucesso!"
    
    def collect_esports_interests(self, main_interests, favorite_teams, favorite_games):
        self.profile_data['esports_interests']['main_interests'] = main_interests
        self.profile_data['esports_interests']['favorite_teams'] = favorite_teams
        self.profile_data['esports_interests']['favorite_games'] = favorite_games
        return "Interesses em e-sports coletados com sucesso!"
    
    def add_event(self, event_name, event_date, event_type):
        self.profile_data['events_attended'].append({
            'name': event_name,
            'date': event_date,
            'type': event_type
        })
        return f"Evento '{event_name}' adicionado com sucesso!"
    
    def add_purchase(self, item, price, date):
        self.profile_data['purchases'].append({
            'item': item,
            'price': float(price),
            'date': date
        })
        return f"Compra '{item}' adicionada com sucesso!"
    
    def save_profile(self, filename='fan_profile.json'):
        with open(filename, 'w') as f:
            json.dump(self.profile_data, f, indent=4)
        return f"Perfil salvo em {filename}"
    
    def load_profile(self, filename='fan_profile.json'):
        try:
            with open(filename, 'r') as f:
                self.profile_data = json.load(f)
            return f"Perfil carregado de {filename}"
        except FileNotFoundError:
            return f"Arquivo {filename} não encontrado"
    
    def display_profile(self):
        output = []
        output.append("=== Resumo do Perfil ===")
        output.append("Informações Pessoais:")
        for k, v in self.profile_data['personal_info'].items():
            output.append(f"- {k}: {v}")
        
        if 'esports_interests' in self.profile_data and self.profile_data['esports_interests']:
            output.append("\nInteresses em E-sports:")
            if 'main_interests' in self.profile_data['esports_interests']:
                output.append("- Principais interesses: " + ", ".join(self.profile_data['esports_interests']['main_interests']))
            if 'favorite_teams' in self.profile_data['esports_interests']:
                output.append("- Times favoritos: " + ", ".join(self.profile_data['esports_interests']['favorite_teams']))
            if 'favorite_games' in self.profile_data['esports_interests']:
                output.append("- Jogos favoritos: " + ", ".join(self.profile_data['esports_interests']['favorite_games']))
        
        output.append("\nEventos Recentes:")
        for event in self.profile_data['events_attended']:
            output.append(f"- {event['name']} ({event['date']}, {event['type']})")
        
        output.append("\nCompras Recentes:")
        total_spent = sum(p['price'] for p in self.profile_data['purchases'])
        for purchase in self.profile_data['purchases']:
            output.append(f"- {purchase['item']}: R$ {purchase['price']:.2f} ({purchase['date']})")
        output.append(f"Total gasto: R$ {total_spent:.2f}")
        
        if 'documents' in self.profile_data and self.profile_data['documents']:
            output.append("\nDocumentos Validados:")
            for doc_name, status in self.profile_data['documents'].items():
                output.append(f"- {doc_name}: {'Validado' if status['valid'] else 'Não validado'}")
        
        if 'social_media' in self.profile_data and self.profile_data['social_media']:
            output.append("\nRedes Sociais Vinculadas:")
            for network, data in self.profile_data['social_media'].items():
                output.append(f"- {network}: @{data['username']}")
        
        if 'esports_profiles' in self.profile_data and self.profile_data['esports_profiles']:
            output.append("\nPerfis de E-sports Validados:")
            for platform, data in self.profile_data['esports_profiles'].items():
                output.append(f"- {platform}: {data.get('label', 'Sem classificação')} ({data.get('score', 0):.2f})")
        
        return "\n".join(output)

# Classe para validação de documentos
class DocumentValidator:
    def __init__(self):
        self.ner_model = SimpleNERModel()
        self.face_model = SimpleFaceModel()
    
    def extract_text_from_image(self, image_data):
        try:
            nparr = np.frombuffer(image_data, np.uint8)
            img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
            
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                          cv2.THRESH_BINARY, 11, 2)
            
            custom_config = r'--oem 3 --psm 6'
            text = pytesseract.image_to_string(thresh, config=custom_config, lang='por')
            
            return text
        except Exception as e:
            print(f"Erro ao processar imagem: {e}")
            return None
    
    def validate_document(self, document_text, user_info):
        entities = self.ner_model(document_text)
        
        doc_info = {}
        current_entity = ""
        
        for entity in entities:
            if entity['entity'] in ['B-PER', 'I-PER']:  
                if entity['entity'] == 'B-PER':
                    if current_entity:
                        doc_info[current_entity] = ' '.join(doc_info[current_entity])
                    current_entity = "name"
                    doc_info[current_entity] = [entity['word']]
                else:
                    if current_entity == "name":
                        doc_info[current_entity].append(entity['word'])
            elif entity['entity'] in ['B-LOC', 'I-LOC']: 
                if entity['entity'] == 'B-LOC':
                    if current_entity:
                        doc_info[current_entity] = ' '.join(doc_info[current_entity])
                    current_entity = "address"
                    doc_info[current_entity] = [entity['word']]
                else:
                    if current_entity == "address":
                        doc_info[current_entity].append(entity['word'])
            elif entity['entity'] in ['B-CPF', 'I-CPF']:  
                if entity['entity'] == 'B-CPF':
                    if current_entity:
                        doc_info[current_entity] = ' '.join(doc_info[current_entity])
                    current_entity = "cpf"
                    doc_info[current_entity] = [entity['word']]
                else:
                    if current_entity == "cpf":
                        doc_info[current_entity].append(entity['word'])
        
        if current_entity:
            doc_info[current_entity] = ' '.join(doc_info[current_entity])
        
        validation_results = {}
        
        if 'name' in doc_info:
            doc_name = ''.join(e for e in doc_info['name'] if e.isalnum() or e.isspace()).lower()
            user_name = ''.join(e for e in user_info['name'] if e.isalnum() or e.isspace()).lower()
            validation_results['name'] = doc_name in user_name or user_name in doc_name
        else:
            validation_results['name'] = False
        
        if 'cpf' in doc_info:
            doc_cpf = ''.join(filter(str.isdigit, doc_info['cpf']))
            user_cpf = ''.join(filter(str.isdigit, user_info['cpf']))
            validation_results['cpf'] = doc_cpf == user_cpf
        else:
            validation_results['cpf'] = False
        
        return validation_results
    
    def compare_faces(self, image1_data, image2_data):
        try:
            img1 = Image.open(io.BytesIO(image1_data))
            img2 = Image.open(io.BytesIO(image2_data))
            
            faces1 = self.face_model(img1)
            faces2 = self.face_model(img2)
            
            if len(faces1) == 0 or len(faces2) == 0:
                return False, "Nenhuma face detectada em uma ou ambas imagens"
            
            return True, "Faces detectadas em ambas imagens (simulação de correspondência)"
        
        except Exception as e:
            return False, f"Erro ao comparar faces: {e}"

class EsportsProfileValidator:
    def __init__(self):
        self.platforms = {
            'FACEIT': 'faceit.com',
            'ESL': 'eslgaming.com',
            'Battlefy': 'battlefy.com',
            'Toornament': 'toornament.com',
            'Strafe': 'strafe.com'
        }
    
    def validate_profile_link(self, url, user_info):
        try:
            domain_valid = any(platform.lower() in url.lower() for platform in self.platforms.values())
            
            profile_name = url.split('/')[-1].lower()
            user_name = user_info['name'].split()[0].lower()
            
            similarity = 0.0
            if user_name in profile_name or profile_name in user_name:
                similarity = 0.7
            elif len(set(user_name) & set(profile_name)) > 2:
                similarity = 0.5
            
            return {
                'domain_valid': domain_valid,
                'name_similarity': similarity,
                'likely_match': domain_valid and similarity > 0.5
            }
        
        except Exception as e:
            print(f"Erro ao validar perfil: {e}")
            return {
                'domain_valid': False,
                'name_similarity': 0,
                'likely_match': False
            }

class SocialMediaIntegrator:
    def __init__(self):
        pass
    
    def link_social_profile(self, network, username, fan_profile):
        if network.lower() == 'twitter':
            twitter_data = {
                'username': username,
                'followers': 450,
                'following': 120,
                'total_tweets': 890,
                'esports_tweets': 156,
                'org_interactions': 42,
                'followed_orgs': ['FURIA', 'LOUD', 'paiN']
            }
            fan_profile.profile_data['social_media']['twitter'] = twitter_data
            return f"Twitter @{username} vinculado com sucesso"
        
        elif network.lower() == 'instagram':
            instagram_data = {
                'username': username,
                'followers': 320,
                'following': 85,
                'total_posts': 76,
                'esports_posts': 23,
                'followed_esports': ['furia', 'loudgg']
            }
            fan_profile.profile_data['social_media']['instagram'] = instagram_data
            return f"Instagram @{username} vinculado com sucesso"
        
        else:
            return f"Rede social {network} não suportada"

def get_image_base64(image_data):
    return base64.b64encode(image_data).decode('utf-8')

class FanAnalyticsDashboard:
    def __init__(self, fan_profile):
        self.fan_profile = fan_profile
    
    def plot_engagement(self):
        plt.style.use('ggplot')
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        fig.suptitle('Análise de Engajamento do Fã', fontsize=16)
        
        if 'main_interests' in self.fan_profile.profile_data['esports_interests']:
            interests = self.fan_profile.profile_data['esports_interests']['main_interests']
            axes[0, 0].bar(range(len(interests)), [1] * len(interests), tick_label=interests)
            axes[0, 0].set_title('Principais Interesses')
            axes[0, 0].set_ylabel('')
            axes[0, 0].set_xlabel('')
        
        if self.fan_profile.profile_data['events_attended']:
            event_types = [e['type'] for e in self.fan_profile.profile_data['events_attended']]
            event_counts = pd.Series(event_types).value_counts()
            axes[0, 1].bar(event_counts.index, event_counts.values)
            axes[0, 1].set_title('Participação em Eventos por Tipo')
            axes[0, 1].set_ylabel('Quantidade')
        
        if self.fan_profile.profile_data['purchases']:
            purchases = []
            for p in self.fan_profile.profile_data['purchases']:
                try:
                    date = datetime.strptime(p['date'], '%d/%m/%Y')
                    purchases.append({
                        'item': p['item'],
                        'price': p['price'],
                        'date': date,
                        'month': date.strftime('%Y-%m')
                    })
                except:
                    pass
            
            if purchases:
                df_purchases = pd.DataFrame(purchases)
                monthly_spending = df_purchases.groupby('month')['price'].sum()
                
                axes[1, 0].plot(monthly_spending.index, monthly_spending.values, marker='o', color='green')
                axes[1, 0].set_title('Gastos Mensais com E-sports')
                axes[1, 0].set_ylabel('Valor (R$)')
                axes[1, 0].tick_params(axis='x', rotation=45)
        
        if 'social_media' in self.fan_profile.profile_data and self.fan_profile.profile_data['social_media']:
            social_metrics = []
            if 'twitter' in self.fan_profile.profile_data['social_media']:
                tw = self.fan_profile.profile_data['social_media']['twitter']
                social_metrics.append(('Twitter', 'Seguidores', tw['followers']))
                social_metrics.append(('Twitter', 'Tweets Esports', tw['esports_tweets']))
            
            if 'instagram' in self.fan_profile.profile_data['social_media']:
                ig = self.fan_profile.profile_data['social_media']['instagram']
                social_metrics.append(('Instagram', 'Seguidores', ig['followers']))
                social_metrics.append(('Instagram', 'Posts Esports', ig['esports_posts']))
            
            if social_metrics:
                platforms = [m[0] for m in social_metrics]
                metrics = [m[1] for m in social_metrics]
                values = [m[2] for m in social_metrics]
                
                colors = ['blue' if m[0] == 'Twitter' else 'purple' for m in social_metrics]
                
                axes[1, 1].bar(range(len(social_metrics)), values, color=colors)
                axes[1, 1].set_title('Engajamento em Redes Sociais')
                axes[1, 1].set_ylabel('Quantidade')
                axes[1, 1].set_xticks(range(len(social_metrics)))
                axes[1, 1].set_xticklabels([f"{p} - {m}" for p, m in zip(platforms, metrics)], rotation=45)
        
        plt.tight_layout()
        plt.subplots_adjust(top=0.9)
        
        buf = BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        
        return buf

def create_interface():
    # Instanciar classes principais
    fan_profile = FanProfile()
    document_validator = DocumentValidator()
    esports_validator = EsportsProfileValidator()
    social_integrator = SocialMediaIntegrator()
    
    output = widgets.Output()
    
    # ====================== Seção de informações pessoais ======================
    name_input = widgets.Text(description='Nome:')
    address_input = widgets.Text(description='Endereço:')
    cpf_input = widgets.Text(description='CPF:')
    birth_date_input = widgets.Text(description='Nascimento:')
    email_input = widgets.Text(description='E-mail:')
    phone_input = widgets.Text(description='Telefone:')
    
    personal_submit = widgets.Button(description='Salvar Dados Pessoais')
    
    @output.capture()
    def on_personal_submit(b):
        result = fan_profile.collect_personal_info(
            name_input.value,
            address_input.value,
            cpf_input.value,
            birth_date_input.value,
            email_input.value,
            phone_input.value
        )
        print(result)
    
    personal_submit.on_click(on_personal_submit)
    
    personal_box = widgets.VBox([
        widgets.HTML("<h3>Informações Pessoais</h3>"),
        name_input, address_input, cpf_input, birth_date_input, email_input, phone_input,
        personal_submit
    ])
    
    # ====================== Seção de interesses em e-sports ======================
    interests_input = widgets.Text(description='Interesses:', 
                                  placeholder='Ex: Jogar, Assistir (separados por vírgula)')
    teams_input = widgets.Text(description='Times:', 
                              placeholder='Ex: FURIA, LOUD (separados por vírgula)')
    games_input = widgets.Text(description='Jogos:', 
                              placeholder='Ex: CS:GO, Valorant (separados por vírgula)')
    
    interests_submit = widgets.Button(description='Salvar Interesses')
    
    @output.capture()
    def on_interests_submit(b):
        interests = [i.strip() for i in interests_input.value.split(',')]
        teams = [t.strip() for t in teams_input.value.split(',')]
        games = [g.strip() for g in games_input.value.split(',')]
        
        result = fan_profile.collect_esports_interests(interests, teams, games)
        print(result)
    
    interests_submit.on_click(on_interests_submit)
    
    interests_box = widgets.VBox([
        widgets.HTML("<h3>Interesses em E-sports</h3>"),
        interests_input, teams_input, games_input,
        interests_submit
    ])
    
    event_name = widgets.Text(description='Nome:')
    event_date = widgets.Text(description='Data:', placeholder='DD/MM/AAAA')
    event_type = widgets.Dropdown(description='Tipo:', options=['presencial', 'online'])
    
    event_submit = widgets.Button(description='Adicionar Evento')
    
    @output.capture()
    def on_event_submit(b):
        result = fan_profile.add_event(
            event_name.value,
            event_date.value,
            event_type.value
        )
        print(result)
        event_name.value = ''
        event_date.value = ''
    
    event_submit.on_click(on_event_submit)
    
    events_box = widgets.VBox([
        widgets.HTML("<h3>Eventos</h3>"),
        event_name, event_date, event_type,
        event_submit
    ])
    
    purchase_item = widgets.Text(description='Item:')
    purchase_price = widgets.FloatText(description='Valor R$:')
    purchase_date = widgets.Text(description='Data:', placeholder='DD/MM/AAAA')
    
    purchase_submit = widgets.Button(description='Adicionar Compra')
    
    @output.capture()
    def on_purchase_submit(b):
        result = fan_profile.add_purchase(
            purchase_item.value,
            purchase_price.value,
            purchase_date.value
        )
        print(result)
        purchase_item.value = ''
        purchase_price.value = 0.0
        purchase_date.value = ''
    
    purchase_submit.on_click(on_purchase_submit)
    
    purchases_box = widgets.VBox([
        widgets.HTML("<h3>Compras</h3>"),
        purchase_item, purchase_price, purchase_date,
        purchase_submit
    ])
    
    doc_upload = FileUpload(accept='image/*', multiple=False, description='Documento:')
    doc_submit = widgets.Button(description='Validar Documento')
    doc_preview = widgets.Output()
    
    @output.capture()
    def on_doc_submit(b):
        if not doc_upload.value:
            print("Por favor, selecione uma imagem de documento para validar")
            return
        
        if not fan_profile.profile_data['personal_info']:
            print("Por favor, preencha as informações pessoais primeiro")
            return
        
        filename, file_info = list(doc_upload.value.items())[0]
        content = file_info['content']
        
        with doc_preview:
            doc_preview.clear_output()
            display(IPImage(data=content))
        
        text = document_validator.extract_text_from_image(content)
        if text:
            results = document_validator.validate_document(text, fan_profile.profile_data['personal_info'])
            
            fan_profile.profile_data['documents'][filename] = {
                'valid': results['name'] and results['cpf'],
                'name_match': results['name'],
                'cpf_match': results['cpf'],
                'text_extracted': text
            }
            
            print(f"Documento {filename} validado:")
            print(f"- Nome corresponde: {'Sim' if results['name'] else 'Não'}")
            print(f"- CPF corresponde: {'Sim' if results['cpf'] else 'Não'}")
            print(f"- Documento válido: {'Sim' if (results['name'] and results['cpf']) else 'Não'}")
        else:
            print("Não foi possível extrair texto do documento")
    
    doc_submit.on_click(on_doc_submit)
    
    document_box = widgets.VBox([
        widgets.HTML("<h3>Validação de Documentos</h3>"),
        doc_upload, doc_submit, doc_preview
    ])
    
    social_network = widgets.Dropdown(
        description='Rede:',
        options=['Twitter', 'Instagram']
    )
    username_input = widgets.Text(description='Usuário:')
    social_submit = widgets.Button(description='Vincular Rede Social')
    
    @output.capture()
    def on_social_submit(b):
        result = social_integrator.link_social_profile(
            social_network.value,
            username_input.value,
            fan_profile
        )
        print(result)
    
    social_submit.on_click(on_social_submit)
    
    social_box = widgets.VBox([
        widgets.HTML("<h3>Redes Sociais</h3>"),
        social_network, username_input, social_submit
    ])
    
    platform_input = widgets.Dropdown(
        description='Plataforma:',
        options=list(esports_validator.platforms.keys())
    )
    profile_url = widgets.Text(description='URL do Perfil:')
    esports_submit = widgets.Button(description='Validar Perfil')
    
    @output.capture()
    def on_esports_submit(b):
        if not fan_profile.profile_data['personal_info']:
            print("Por favor, preencha as informações pessoais primeiro")
            return
        
        result = esports_validator.validate_profile_link(
            profile_url.value,
            fan_profile.profile_data['personal_info']
        )
        
        platform = platform_input.value
        fan_profile.profile_data['esports_profiles'][platform] = result
        
        print(f"Perfil {platform} validado:")
        print(f"- Domínio válido: {'Sim' if result['domain_valid'] else 'Não'}")
        print(f"- Similaridade de nome: {result['name_similarity']:.2f}")
        print(f"- Provável correspondência: {'Sim' if result['likely_match'] else 'Não'}")
    
    esports_submit.on_click(on_esports_submit)
    
    esports_box = widgets.VBox([
        widgets.HTML("<h3>Perfis de E-sports</h3>"),
        platform_input, profile_url, esports_submit
    ])
    
    display_profile_btn = widgets.Button(description='Exibir Perfil')
    
    @output.capture()
    def on_display_profile(b):
        print(fan_profile.display_profile())
    
    display_profile_btn.on_click(on_display_profile)
    
    save_profile_btn = widgets.Button(description='Salvar Perfil')
    load_profile_btn = widgets.Button(description='Carregar Perfil')
    
    @output.capture()
    def on_save_profile(b):
        result = fan_profile.save_profile()
        print(result)
    
    @output.capture()
    def on_load_profile(b):
        result = fan_profile.load_profile()
        print(result)
        if 'personal_info' in fan_profile.profile_data:
            name_input.value = fan_profile.profile_data['personal_info'].get('name', '')
            address_input.value = fan_profile.profile_data['personal_info'].get('address', '')
            cpf_input.value = fan_profile.profile_data['personal_info'].get('cpf', '')
            birth_date_input.value = fan_profile.profile_data['personal_info'].get('birth_date', '')
            email_input.value = fan_profile.profile_data['personal_info'].get('email', '')
            phone_input.value = fan_profile.profile_data['personal_info'].get('phone', '')
        
        if 'esports_interests' in fan_profile.profile_data:
            interests_input.value = ', '.join(fan_profile.profile_data['esports_interests'].get('main_interests', []))
            teams_input.value = ', '.join(fan_profile.profile_data['esports_interests'].get('favorite_teams', []))
            games_input.value = ', '.join(fan_profile.profile_data['esports_interests'].get('favorite_games', []))
    
    save_profile_btn.on_click(on_save_profile)
    load_profile_btn.on_click(on_load_profile)
    
    profile_box = widgets.VBox([
        widgets.HTML("<h3>Gerenciamento de Perfil</h3>"),
        widgets.HBox([display_profile_btn, save_profile_btn, load_profile_btn])
    ])
    
    generate_analytics_btn = widgets.Button(description='Gerar Análise')
    analytics_output = widgets.Output()
    
    @output.capture()
    def on_generate_analytics(b):
        print("Gerando análise de engajamento do fã...")
        dashboard = FanAnalyticsDashboard(fan_profile)
        image_data = dashboard.plot_engagement()
        
        with analytics_output:
            analytics_output.clear_output()
            display(IPImage(data=image_data.getvalue()))
        
        print("Análise gerada com sucesso!")
        
        engagement_score = calculate_engagement_score(fan_profile)
        print(f"\nPontuação de Engajamento: {engagement_score}/100")
        print(f"Classificação: {get_engagement_tier(engagement_score)}")
        
        generate_recommendations(fan_profile, engagement_score)
    
    generate_analytics_btn.on_click(on_generate_analytics)
    
    analytics_box = widgets.VBox([
        widgets.HTML("<h3>Análise de Engajamento</h3>"),
        generate_analytics_btn,
        analytics_output
    ])
    
    def calculate_engagement_score(fan_profile):
        score = 0
        
        if fan_profile.profile_data['personal_info']:
            score += 10
        
        if 'esports_interests' in fan_profile.profile_data:
            interests = fan_profile.profile_data['esports_interests']
            if 'main_interests' in interests and interests['main_interests']:
                score += min(len(interests['main_interests']), 3) * 5
            if 'favorite_teams' in interests and interests['favorite_teams']:
                score += min(len(interests['favorite_teams']), 3) * 5
            if 'favorite_games' in interests and interests['favorite_games']:
                score += min(len(interests['favorite_games']), 3) * 5
        
        score += min(len(fan_profile.profile_data['events_attended']), 5) * 3
        
        score += min(len(fan_profile.profile_data['purchases']), 5) * 2
        
        if 'documents' in fan_profile.profile_data:
            valid_docs = sum(1 for doc in fan_profile.profile_data['documents'].values() if doc.get('valid', False))
            score += valid_docs * 5
        
        if 'social_media' in fan_profile.profile_data:
            score += len(fan_profile.profile_data['social_media']) * 5
        
        if 'esports_profiles' in fan_profile.profile_data:
            valid_profiles = sum(1 for profile in fan_profile.profile_data['esports_profiles'].values() 
                                if profile.get('likely_match', False))
            score += valid_profiles * 5
        
        return min(score, 100)  
    
    def get_engagement_tier(score):
        if score >= 90:
            return "Fã Superfã"
        elif score >= 70:
            return "Fã Dedicado"
        elif score >= 50:
            return "Fã Engajado"
        elif score >= 30:
            return "Fã Casual"
        else:
            return "Fã Iniciante"
    
    @output.capture()
    def generate_recommendations(fan_profile, engagement_score):
        print("\n=== Recomendações Personalizadas ===")
        
        if 'esports_interests' in fan_profile.profile_data:
            interests = fan_profile.profile_data['esports_interests']
            favorite_games = interests.get('favorite_games', [])
            favorite_teams = interests.get('favorite_teams', [])
            
            if favorite_games:
                print(f"Baseado em seu interesse em {', '.join(favorite_games)}:")
                for game in favorite_games[:2]:  
                    if game.lower() == 'cs:go' or game.lower() == 'csgo' or game.lower() == 'counter-strike':
                        print("- Recomendamos seguir os maiores torneios de CS:GO como ESL Pro League e BLAST Premier")
                        print("- Campeonatos nacionais como CBCS também são uma ótima opção para conhecer novos talentos")
                    elif game.lower() == 'valorant':
                        print("- Fique atento aos próximos Champions e Masters de Valorant")
                        print("- O VCT Brasil é uma ótima porta de entrada para o cenário nacional")
                    elif game.lower() == 'league of legends' or game.lower() == 'lol':
                        print("- Acompanhe o CBLOL e torneios internacionais como MSI e Worlds")
                        print("- Streams de jogadores profissionais podem ajudar a melhorar suas habilidades")
                    else:
                        print(f"- Procure torneios de {game} para acompanhar presencialmente")
                        print(f"- Siga jogadores profissionais de {game} para ficar atualizado")
            
            if favorite_teams:
                print(f"\nBaseado em seu interesse nos times {', '.join(favorite_teams)}:")
                for team in favorite_teams[:2]:  
                    if team.lower() == 'furia' or team.lower() == 'furia esports':
                        print("- Fique atento às promoções exclusivas da loja oficial da FURIA")
                        print("- Acompanhe os jogos da FURIA Academy para conhecer novos talentos")
                    elif team.lower() == 'loud':
                        print("- Não perca os próximos eventos presenciais com a presença da LOUD")
                        print("- Fique atento às colaborações da LOUD com marcas para produtos exclusivos")
                    elif team.lower() == 'pain' or team.lower() == 'pain gaming':
                        print("- Participe dos eventos da paiN com encontro de fãs")
                        print("- Siga os jogadores da paiN nas redes sociais para conteúdo exclusivo")
                    else:
                        print(f"- Inscreva-se na newsletter do {team} para ficar por dentro das novidades")
                        print(f"- Procure grupos de fãs do {team} para interagir com outros torcedores")
        
        print("\nBaseado no seu nível de engajamento:")
        
        if engagement_score >= 70:
            print("- Você já é um fã dedicado! Considere participar de programas de embaixadores")
            print("- Eventos VIP e Meet & Greet com jogadores seriam ótimas experiências para você")
        elif engagement_score >= 50:
            print("- Participe de mais eventos presenciais para aumentar seu engajamento")
            print("- Considere seguir mais jogadores e times nas redes sociais")
        else:
            print("- Comece seguindo seus times favoritos nas redes sociais")
            print("- Experimente assistir a um evento presencial de e-sports na sua região")
        
        print("\nPara melhorar seu perfil de fã:")
        
        if 'documents' not in fan_profile.profile_data or not fan_profile.profile_data['documents']:
            print("- Valide seus documentos para ter acesso a promoções exclusivas")
        
        if 'social_media' not in fan_profile.profile_data or len(fan_profile.profile_data['social_media']) < 2:
            print("- Vincule mais redes sociais para melhorar seu engajamento digital")
        
        if 'esports_profiles' not in fan_profile.profile_data or not fan_profile.profile_data['esports_profiles']:
            print("- Vincule seus perfis de plataformas de e-sports como FACEIT ou ESL")
        
        if len(fan_profile.profile_data['events_attended']) < 3:
            print("- Participe de mais eventos para aumentar seu histórico de engajamento")
    
    tabs = widgets.Tab()
    tabs.children = [
        widgets.VBox([personal_box, interests_box]),
        widgets.VBox([events_box, purchases_box]),
        widgets.VBox([document_box]),
        widgets.VBox([social_box, esports_box]),
        widgets.VBox([profile_box, analytics_box]),
    ]
    
    tabs.set_title(0, 'Dados Básicos')
    tabs.set_title(1, 'Eventos e Compras')
    tabs.set_title(2, 'Documentos')
    tabs.set_title(3, 'Redes e Perfis')
    tabs.set_title(4, 'Análise')
    
    interface = widgets.VBox([
        widgets.HTML("<h1>Sistema de Gestão de Perfil de Fã</h1>"),
        tabs,
        output
    ])
    
    return interface

def start_app():
    interface = create_interface()
    display(interface)
    print("Bem-vindo ao Sistema de Gestão de Perfil de Fã!")
    print("Utilize as abas acima para navegar entre as diferentes seções do sistema.")
    print("Para começar, preencha seus dados pessoais e interesses na aba 'Dados Básicos'.")

if __name__ == "__main__":
    start_app()

class AIRecommendationEngine:
    def __init__(self):
        pass
    
    def generate_personalized_recommendations(self, fan_profile):
        recommendations = []
        
        if 'esports_interests' in fan_profile.profile_data:
            interests = fan_profile.profile_data['esports_interests']
            
            if 'favorite_games' in interests and interests['favorite_games']:
                games = interests['favorite_games']
                for game in games:
                    if game.lower() in ['cs:go', 'csgo', 'counter-strike']:
                        recommendations.append({
                            'type': 'evento',
                            'title': 'ESL Pro League Season 25',
                            'description': 'Maior liga de CS:GO do mundo com as melhores equipes',
                            'date': '15/06/2025',
                            'relevance': 0.9
                        })
                    elif game.lower() == 'valorant':
                        recommendations.append({
                            'type': 'evento',
                            'title': 'VCT Champions 2025',
                            'description': 'Campeonato mundial de Valorant',
                            'date': '10/08/2025',
                            'relevance': 0.9
                        })
                    elif game.lower() in ['league of legends', 'lol']:
                        recommendations.append({
                            'type': 'evento',
                            'title': 'CBLOL Split 2 2025',
                            'description': 'Campeonato Brasileiro de League of Legends',
                            'date': '22/05/2025',
                            'relevance': 0.85
                        })
            
            if 'favorite_teams' in interests and interests['favorite_teams']:
                teams = interests['favorite_teams']
                for team in teams:
                    if team.lower() == 'furia':
                        recommendations.append({
                            'type': 'produto',
                            'title': 'Camisa Oficial FURIA 2025',
                            'description': 'Nova camisa oficial com tecnologia de resfriamento',
                            'price': 249.90,
                            'relevance': 0.95
                        })
                    elif team.lower() == 'loud':
                        recommendations.append({
                            'type': 'produto',
                            'title': 'Moletom LOUD Edição Limitada',
                            'description': 'Moletom exclusivo com design dos jogadores',
                            'price': 299.90,
                            'relevance': 0.9
                        })
        
        if fan_profile.profile_data['purchases']:
            purchase_items = [p['item'].lower() for p in fan_profile.profile_data['purchases']]
            
            if any('camisa' in item for item in purchase_items):
                recommendations.append({
                    'type': 'produto',
                    'title': 'Kit Esportivo Limited Edition',
                    'description': 'Kit exclusivo com camisa, boné e máscara',
                    'price': 399.90,
                    'relevance': 0.85
                })
            
            if any('ingresso' in item for item in purchase_items):
                recommendations.append({
                    'type': 'evento',
                    'title': 'GameCon Brasil 2025',
                    'description': 'Maior evento de games do Brasil com área de e-sports',
                    'date': '30/07/2025',
                    'relevance': 0.8
                })
        
        if fan_profile.profile_data['events_attended']:
            event_types = [e['type'] for e in fan_profile.profile_data['events_attended']]
            
            if 'presencial' in event_types:
                recommendations.append({
                    'type': 'evento',
                    'title': 'E-Sports Experience 2025',
                    'description': 'Evento com experiências imersivas e encontro com jogadores',
                    'date': '25/09/2025',
                    'relevance': 0.9
                })
            
            if 'online' in event_types:
                recommendations.append({
                    'type': 'digital',
                    'title': 'Pacote Digital Premium',
                    'description': 'Acesso exclusivo a conteúdos digitais e transmissões',
                    'price': 149.90,
                    'relevance': 0.75
                })
        
        recommendations.sort(key=lambda x: x['relevance'], reverse=True)
        
        return recommendations[:5]  

def display_recommendations(recommendations):
    if not recommendations:
        return "Nenhuma recomendação disponível no momento."
    
    output = []
    output.append("=== Recomendações Personalizadas ===")
    
    for i, rec in enumerate(recommendations, 1):
        output.append(f"\n{i}. {rec['title']}")
        output.append(f"   {rec['description']}")
        
        if 'date' in rec:
            output.append(f"   Data: {rec['date']}")
        
        if 'price' in rec:
            output.append(f"   Preço: R$ {rec['price']:.2f}")
        
        output.append(f"   Relevância: {int(rec['relevance'] * 100)}%")
    
    return "\n".join(output)

if __name__ == "__main__":
    start_app()

Bem-vindo ao Sistema de Gestão de Perfil de Fã!
Utilize as abas acima para navegar entre as diferentes seções do sistema.
Para começar, preencha seus dados pessoais e interesses na aba 'Dados Básicos'.


Bem-vindo ao Sistema de Gestão de Perfil de Fã!
Utilize as abas acima para navegar entre as diferentes seções do sistema.
Para começar, preencha seus dados pessoais e interesses na aba 'Dados Básicos'.
